In [11]:
!mkdir -p drive

datalab  drive


In [20]:
#making directory for test train datasets
!mkdir -p drive/Test
!mkdir -p drive/Train
!ls

datalab  drive	Test  Train


In [4]:
#uploading datasets
from google.colab import files
uploaded = files.upload()

#!unzip Data.zip for unzipig

Saving Data.zip to Data.zip


In [0]:
#/content/Data/Train/train_dog      train for dog
#/content/Data/Train/train_cat      train for cat

#/content/Data/train_dog -> /content/Data/Test/test_dog
#/content/Data/train_cat -> /content/Data/Test/test_cat

#test1 must name photo

In [0]:
size = (128,128)

#all img in dir '.jpg'
def get_imlist(path):
  return [os.path.join(path,f) for f in os.listdir(path) if f.endswith('.jpg')]

#change size of photo
def preapre(size, path):
  imlist = get_imlist(path)
  for img in imlist:
    im = Image.open(img)
    im.thumbnail(size)
    im.save(img, "JPEG")

In [0]:
import os

coef_for_test = 0.2
path = ""
fr = ""
x_test_path = ""
x_train_path = ""

imlist = get_imlist(path)
split(fr, x_test_path, x_train_path, imlist, coef_for_test)

In [0]:
def split(fr, x_test_path, x_train_path, imlist, coef_for_test=0.2):
  '''
      fr - string path to directory where located photos
      x_test_path - string path where will be saved test set like /content/Data/Test/test_dog
      x_train_path - string path where will be saved train set like /content/Data/Train/train_dog
      imlist  - lst[str] - all photo in dir
      coef_for_test - float - percent for spliting data set to test train set default = 0.2
      
      moving images to x_test_path, x_train_path directories spliting them in proportions coef_for_test / 1 - coef_for_test
  '''
  count_test = len(imlist) * coef_for_test
  for index, im in enumerate(imlist):
    if index < count_test:
      os.rename(os.path.join(fr, im), x_test_path)
    else:
      os.rename(os.path.join(fr, im), x_train_path)
  

In [0]:
#importing packages and libraries
from keras.models import Sequential
from keras.layers import Convolution2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.layers import Dense
from keras.layers import Dropout
from keras.preprocessing.image import ImageDataGenerator
import pandas as pd
import numpy as np

In [0]:
# architecture of model
def model(test_path, train_path, input_shape):
    classifier = Sequential()
    
    classifier.add(Convolution2D(32, (3, 3), input_shape=input_shape, activation='relu', dim_ordering="tf"))
    classifier.add(MaxPooling2D(pool_size=(2, 2), dim_ordering="tf"))
    
    classifier.add(Convolution2D(32, (3, 3), input_shape=(125, 125, 3), activation='relu', dim_ordering="tf"))
    classifier.add(MaxPooling2D(pool_size=(2, 2), dim_ordering="tf"))
    
    classifier.add(Convolution2D(64, (3, 3), input_shape=(125, 125, 3), activation='relu', dim_ordering="tf"))
    classifier.add(MaxPooling2D(pool_size=(2, 2), dim_ordering="tf"))
    
    
    classifier.add(Flatten())
    classifier.add(Dense(64, activation='relu'))
    classifier.add(Dropout(0.5))
    classifier.add(Dense(1, activation='sigmoid'))
    
    classifier.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics=['accuracy'])
    
    train_datagen = ImageDataGenerator(
            rescale=1./255,
            shear_range=0.2,
            zoom_range=0.2,
            horizontal_flip=True)
    
    test_datagen = ImageDataGenerator(rescale=1./255)
    
    train_set = train_datagen.flow_from_directory(
            train_path,
            target_size=(125, 125),
            batch_size=32,
            class_mode='binary')
    
    test_set = test_datagen.flow_from_directory(
            test_path,
            target_size=(125, 125),
            batch_size=32,
            class_mode='binary')
    
    classifier.fit_generator(epochs=30,validation_steps=32, 
            generator=train_set,
            validation_data=test_set)
    return classifier

In [220]:
test_path = ""
train_path = ""
input_shape = ()
model = model(test_path, train_path, input_shape)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(32, (3, 3), input_shape=(125, 125,..., activation="relu", data_format="channels_last")`
  after removing the cwd from sys.path.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: UserWarning: Update your `MaxPooling2D` call to the Keras 2 API: `MaxPooling2D(pool_size=(2, 2), data_format="channels_last")`
  """
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:7: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(32, (3, 3), input_shape=(125, 125,..., activation="relu", data_format="channels_last")`
  import sys
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:8: UserWarning: Update your `MaxPooling2D` call to the Keras 2 API: `MaxPooling2D(pool_size=(2, 2), data_format="channels_last")`
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:10: UserWarning: Update your `Conv2D` call to the Keras 2

Found 20000 images belonging to 2 classes.
Found 5000 images belonging to 2 classes.
Epoch 1/30
568/625 [==========================>...] - ETA: 9s - loss: 0.6589 - acc: 0.6034

625/625 [==============================] - 104s 166ms/step - loss: 0.6558 - acc: 0.6069 - val_loss: 0.5915 - val_acc: 0.6934
Epoch 2/30
442/625 [====================>.........] - ETA: 29s - loss: 0.5819 - acc: 0.6973

625/625 [==============================] - 103s 164ms/step - loss: 0.5740 - acc: 0.7025 - val_loss: 0.5265 - val_acc: 0.7471
Epoch 3/30
395/625 [=================>............] - ETA: 37s - loss: 0.5300 - acc: 0.7390

625/625 [==============================] - 103s 164ms/step - loss: 0.5179 - acc: 0.7473 - val_loss: 0.5687 - val_acc: 0.7080
Epoch 4/30
378/625 [=================>............] - ETA: 40s - loss: 0.4792 - acc: 0.7700

...

625/625 [==============================] - 103s 165ms/step - loss: 0.2426 - acc: 0.9002 - val_loss: 0.2999 - val_acc: 0.8818
Epoch 27/30
365/625 [================>.............] - ETA: 42s - loss: 0.2366 - acc: 0.9002

625/625 [==============================] - 103s 165ms/step - loss: 0.2400 - acc: 0.9002 - val_loss: 0.2749 - val_acc: 0.8955
Epoch 28/30
365/625 [================>.............] - ETA: 41s - loss: 0.2407 - acc: 0.9022

625/625 [==============================] - 102s 163ms/step - loss: 0.2420 - acc: 0.9018 - val_loss: 0.2791 - val_acc: 0.8838
Epoch 29/30
365/625 [================>.............] - ETA: 42s - loss: 0.2289 - acc: 0.9033

625/625 [==============================] - 103s 165ms/step - loss: 0.2303 - acc: 0.9036 - val_loss: 0.2914 - val_acc: 0.8799
Epoch 30/30
365/625 [================>.............] - ETA: 42s - loss: 0.2296 - acc: 0.9042

625/625 [==============================] - 103s 165ms/step - loss: 0.2320 - acc: 0.9035 - val_loss: 0.2691 - val_acc: 0.8848


In [0]:
model.save_weights('model_wieghts.h5')
model.save('model_keras.h5')

In [0]:
test_datagen = ImageDataGenerator(rescale=1. / 255)
test_generator = test_datagen.flow_from_directory('test', (125, 125), class_mode=None)
prediction_probabilities = model.predict_generator(test_generator, verbose=1)

In [0]:
prediction_probabilities = prediction_probabilities.reshape(12500,)
prediction_probabilities

for i in range(len(prediction_probabilities)):
  if prediction_probabilities[i] > 0.5:
    prediction_probabilities[i] = 1
  else:
    prediction_probabilities[i] = 0
prediction_probabilities

In [0]:
counter = range(1, len(imlist) + 1)
solution = pd.DataFrame({"id": counter, "label":prediction_probabilities})
cols = ['label']

for col in cols:
  solution[col] = solution[col].map(lambda x: str(x).lstrip('[').rstrip(']')).astype(float)

solution.to_csv("dogsVScats.csv", index = False)

In [0]:
files.download('dogsVScats.csv')